In [13]:
import pandas as pd
import sqlite3
import seaborn as sb
import matplotlib as plt

In [14]:
conn = sqlite3.connect('space_missions.sqlite')
cur = conn.cursor()

conn2 = sqlite3.connect('space_missions_insights.sqlite') 
cur = conn.cursor()

In [15]:
# With the obtained previous data of launches, we query top 5 countries with successful launches to obtain the launch location
q = '''
SELECT Company, Country, Location, Date, Rocket, Mission
FROM records
WHERE (Country = 'USA' OR Country = 'Russia' OR Country = 'Kazakhstan' OR Country = 'China' OR Country = 'France');
'''
df = pd.read_sql_query(q,conn)
df.to_sql('top5_launc_loc', conn2, if_exists='replace', index=False)

df.head(4)

,Company,Country,Location,Date,Rocket,Mission
0,RVSN USSR,Kazakhstan,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",04/10/1957,Sputnik 8K71PS,Sputnik-1
1,RVSN USSR,Kazakhstan,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",03/11/1957,Sputnik 8K71PS,Sputnik-2
2,US Navy,USA,"LC-18A, Cape Canaveral AFS, Florida, USA",06/12/1957,Vanguard,Vanguard TV3
3,AMBA,USA,"LC-26A, Cape Canaveral AFS, Florida, USA",01/02/1958,Juno I,Explorer 1


In [16]:
q2 ='''
SELECT SUBSTR(Date, -4) AS año, COUNT(*) AS launches_per_year
FROM records
GROUP BY SUBSTR(Date, -4)
ORDER BY launches_per_year DESC
LIMIT 5;
'''

df2 = pd.read_sql_query(q2,conn)
df.to_sql('launches_per_year', conn2, if_exists='replace', index=False)
df2

,año,launches_per_year
0,2021,157
1,2020,119
2,1971,119
3,2018,117
4,1977,114


In [17]:
# Determining the active & retired rockets
q3= '''
SELECT Country, 
       SUM(CASE WHEN RocketStatus = 'Active' THEN 1 ELSE 0 END) AS Active,
       SUM(CASE WHEN RocketStatus = 'Retired' THEN 1 ELSE 0 END) AS Retired
FROM records
GROUP BY Country
ORDER BY Country;
'''
df3 = pd.read_sql_query(q3, conn)
df3.to_sql('Act_Ret_rockets', conn2, if_exists='replace', index=False)

df3

,Country,Active,Retired
0,Australia,0,6
1,Barents Sea,0,3
2,Brazil,0,3
3,China,303,62
4,France,128,190
5,Gran Canaria,2,0
6,India,56,26
7,Iran,12,4
8,Israel,5,6
9,Japan,42,88


In [18]:
# A filter with some rocket info

q4 = '''
SELECT Country, Company, strftime('%Y', Date) AS Year, Rocket, MissionStatus, RocketStatus
FROM records
'''
df4 = pd.read_sql_query(q4, conn)
df4.to_sql('gral_rockt_info', conn2, if_exists='replace',index=False)
df4.tail(5)

,Country,Company,Year,Rocket,MissionStatus,RocketStatus
4625,USA,SpaceX,None,Falcon 9 Block 5,Success,Active
4626,China,CASC,None,Long March 5B,Success,Active
4627,USA,SpaceX,None,Falcon 9 Block 5,Success,Active
4628,China,CAS Space,None,Zhongke-1A,Success,Active
4629,China,CASC,None,Long March 2D,Success,Active
